# Fine & Golden Autoscan v21
*All‑in‑one Colab notebook – updated 2025-07-14*

### What’s new in v21
1. **Self‑contained** – no prior chat context required.
2. **Multi‑constant delta engine** – analyse proximity to α⁻¹, φ, √2, e, and any custom reference.
3. **Pluggable clustering** – choose DBSCAN / MeanShift with metric & bandwidth knobs.
4. **Change detector** – automatically flags new slice values/clusters vs a previous version.
5. **Clean outputs** – summary table + CSV exports.


In [21]:

# ======================= CONFIGURATION =============================
VERSION        = 21      # current run tag
PREV_VERSION   = 20      # set to None if no previous results to diff
DATA_DIR       = ''      # <-- point to folder with Excel files & outputs
NUM_COL        = 'slice' # numeric column to analyse
DIST_METRIC    = 'euclidean'  # 'euclidean', 'manhattan', 'cosine', ...
BANDWIDTH      = 0.001   # ε / bandwidth for clustering
NEW_THRESHOLD  = 1e-12   # consider value new if |x - prev_x| > threshold
# ===================================================================


In [22]:

# Mount Google Drive if in Colab (optional)
try:
    import google.colab, os
    from google.colab import drive
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')
        print('✔ Drive mounted.')
except ModuleNotFoundError:
    pass

# Manual upload fallback
# from google.colab import files; files.upload()


In [23]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN, MeanShift
from pathlib import Path


In [24]:

ALPHA_INV = 137.035999084
PHI       = (1 + 5**0.5) / 2
SQRT2     = 2**0.5
EULER_E   = np.e

REFS = {'alpha_inv': ALPHA_INV, 'phi': PHI, 'sqrt2': SQRT2, 'e': EULER_E}

def delta(x, ref):
    """Absolute fractional difference between x and ref."""
    return abs(x - ref) / ref

def load_plateau(version:int):
    raw = Path(DATA_DIR) / f'plateaus_raw_v{version}.xlsx'
    ops = Path(DATA_DIR) / f'plateaus_ops_v{version}.xlsx'
    if not raw.exists() or not ops.exists():
        raise FileNotFoundError(f'Missing Excel files for v{version}.')
    return pd.read_excel(raw), pd.read_excel(ops)

def save_csv(df:pd.DataFrame, name:str):
    out = Path(DATA_DIR) / name
    df.to_csv(out, index=False)
    print(f'✔ Saved {out.name}')


In [25]:

raw_df, ops_df = load_plateau(VERSION)
print(f'Loaded raw {raw_df.shape}, ops {ops_df.shape}')


FileNotFoundError: Missing Excel files for v21.

In [ ]:

# Compute deltas for all references
for key, ref in REFS.items():
    ops_df[f'delta_{key}'] = ops_df[NUM_COL].apply(lambda x: delta(x, ref))

# Preview
ops_df.head()


In [ ]:

vals = ops_df[[NUM_COL]].values
if DIST_METRIC in {'euclidean', 'manhattan'}:
    model = DBSCAN(eps=BANDWIDTH, metric=DIST_METRIC, min_samples=2).fit(vals)
else:
    model = MeanShift(bandwidth=BANDWIDTH).fit(vals)
ops_df['cluster'] = model.labels_
print('Cluster counts:\n', ops_df['cluster'].value_counts())


In [ ]:

new_entries = pd.DataFrame()
if PREV_VERSION is not None:
    prev_path = Path(DATA_DIR) / f'ops_with_clusters_v{PREV_VERSION}.csv'
    if prev_path.exists():
        prev_df = pd.read_csv(prev_path)
        prev_vals = set(np.round(prev_df[NUM_COL], 12))
        curr_vals = set(np.round(ops_df[NUM_COL], 12))
        new_vals = [v for v in curr_vals if all(abs(v - pv) > NEW_THRESHOLD for pv in prev_vals)]
        new_entries = ops_df[np.isin(np.round(ops_df[NUM_COL], 12), new_vals)]
        print(f'▲ Detected {len(new_entries)} new slice values vs v{PREV_VERSION}')
    else:
        print(f'No previous result file at {prev_path.name}')
else:
    print('PREV_VERSION set to None – skipping diff.')


In [ ]:

plt.figure(figsize=(10,5))
plt.scatter(ops_df.index, ops_df[NUM_COL], c=ops_df['cluster'])
plt.title('Clustered slices (v%d)' % VERSION)
plt.xlabel('Index'); plt.ylabel(NUM_COL)
plt.show()


In [ ]:

save_csv(ops_df, f'ops_with_clusters_v{VERSION}.csv')
if not new_entries.empty:
    save_csv(new_entries, f'new_entries_vs_v{PREV_VERSION}_to_v{VERSION}.csv')
